In [1]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(''), '..'))

import numpy
import pandas
import custom
import pickle
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from IPython.display import display


In [2]:
##데이터 불러오기
df = pandas.read_csv("data/simple_movie_data.csv", encoding="UTF8", index_col=0)
display(df)

with open("data/vector.pkl", mode="rb") as f:
    vec = pickle.load(f)

print(len(vec))

,quote,scoreSentiment,word_len
22,it has not aged as well as some oscar winners ...,1,20
24,if you have not seen it do,1,7
25,you ca not take it with you was a tonic to a d...,1,20
27,this movie is pure soap opera but what a soap ...,1,11
32,this is very nearly a great movie it comes clo...,1,19
...,...,...,...
967548,sorry not sorry skates over much of this recen...,0,14
967551,in spite of its conclusion N it N s a promisin...,1,19
967554,i appreciate that both girls came to understan...,1,18
967557,N jackie N chan is at his damn finest N,1,10


123281


In [3]:
##데이터 전처리

sentences = df["quote"].values.tolist()
t = df["scoreSentiment"].values.tolist()
word_len = df["word_len"].max()

x = []
for s in sentences :
    temp = []
    words = custom.text_preprocess(s).split()
    for i in range(word_len - len(words)) :
        temp.append(vec["<pad>"])
    for i in range(len(words)) :
        if words[i] not in vec :
            temp.append(vec["<unk>"])
            continue
        temp.append(vec[words[i]])
    x.append(temp)
print(len(x))

test_size = int(len(x) * 0.2)
print(test_size)

372505
74501


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

##DataLoader 생성
tensor_x = torch.tensor(x, dtype = torch.float, device = device)
tensor_t = torch.tensor(t, dtype = torch.long, device = device)
zip_list = list(zip(tensor_x, tensor_t))
val_dataloader = DataLoader(zip_list[:test_size], batch_size= 5000, shuffle= False)
dataloader = DataLoader(zip_list[test_size:], batch_size= 500, shuffle= True)



In [5]:
##AI 만들기
#함수 선언
class NN(nn.Module) :
    def __init__(self) :
        super().__init__()
        self.rnn = nn.RNN(300,10,batch_first = True,device = device)
        self.f = nn.Sequential(
            nn.Linear(10,2),
            nn.ReLU()
        )
    def forward(self, x) :
        x, h = self.rnn(x)
        x = x[:,-1,:]
        x = self.f(x)
        return x
F = NN()


In [9]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(F.parameters(), lr = 0.25)
epoch = 50

for e in range(epoch) :
    loss_sum = 0
    for x, t in dataloader :
    # x, t 입력
    # y = F(x)
        y = F(x)
    # 손실함수
        loss = loss_function(y,t)
        loss_sum += loss
    # 최적화함수
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"epoch {e+1} || loss {loss_sum / len(dataloader)}")

epoch 1 || loss 0.5853430032730103
epoch 2 || loss 0.526194155216217
epoch 3 || loss 0.4910239577293396
epoch 4 || loss 0.475610613822937
epoch 5 || loss 0.4663142263889313
epoch 6 || loss 0.4598471224308014
epoch 7 || loss 0.45318087935447693
epoch 8 || loss 0.448007732629776
epoch 9 || loss 0.4434940814971924
epoch 10 || loss 0.44010797142982483
epoch 11 || loss 0.43617600202560425
epoch 12 || loss 0.4340313673019409
epoch 13 || loss 0.4297901391983032
epoch 14 || loss 0.4279608428478241
epoch 15 || loss 0.42714813351631165
epoch 16 || loss 0.423537015914917
epoch 17 || loss 0.4209586977958679
epoch 18 || loss 0.41879287362098694
epoch 19 || loss 0.416061669588089
epoch 20 || loss 0.4143688976764679
epoch 21 || loss 0.4133390188217163
epoch 22 || loss 0.41237738728523254
epoch 23 || loss 0.41026589274406433
epoch 24 || loss 0.409969300031662
epoch 25 || loss 0.40773916244506836
epoch 26 || loss 0.4079040586948395
epoch 27 || loss 0.4065694510936737
epoch 28 || loss 0.4036673903465271

In [10]:
cnt = 0
y =F(val_x)
for i in range(val_x.shape[0]) :
    if torch.argmax(y[i]) == val_t[i] :
        cnt += 1

print("possiblity : %f" %(cnt / val_x.shape[0]))

possiblity : 0.817466


In [11]:
torch.save(F, "simple_movie_RNN.pt")